In [1]:
import sqlite3
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np 

# Scrapping Apps and Metascore from metacritic:

In [25]:
def meta_url(page_num=0):
    return f"https://www.metacritic.com/browse/games/release-date/available/ios/metascore?page={page_num}"

def get_meta_score(total_page = 1):
    meta_score = []
    for page in range(total_page):
        # headers to prevent 403 denial
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
        response = requests.get(meta_url(page),headers = headers)
        soup = BeautifulSoup(response.text,'html.parser')
        # findall tables with app and scores
        tables = soup.find_all("table",{"class":"clamp-list"})
        for table in tables:
            rows = table.find_all("tr")
            for row in rows:
                # skip empty rows "tr":"spacer":(only process a-tag with class=title)
                if row.find("a",{"class":"title"}):
                    name = row.find("a",{"class":"title"}).find("h3").get_text()
                    score = row.find("a",{"class":"metascore_anchor"}).get_text().replace("\n","")
                    meta_score.append((name,score))
    return meta_score

# SQLite DataBase Related Functions:

In [16]:
# Database Related Functions:
def create_connection():
    conn = sqlite3.connect("metascore.db")
    cur = conn.cursor()
    return conn,cur

# create 'meta_score' table (with data we scrap in the first time) 
def create_meta_data_table(meta_scores):
    conn, cur = create_connection()
    # Create a brand new "meta_score" table in db
    cur.execute('DROP TABLE meta_score');
    cur.execute('CREATE TABLE meta_score (name TEXT, score INTEGER)');
    cur.executemany('INSERT INTO meta_score VALUES (?,?)',meta_scores);
    conn.commit()
    # Check table content:
    # cur.execute('SELECT * FROM meta_score')
    # print(cur.fetchall())
    conn.close()

# Insert to 'meta_score' table, in case more app data needed
def insert_meta_data(meta):
    conn,cur = create_connection()
    cur.execute('INSERT INTO meta_score VALUES (?,?)',meta)
    conn.comit()
    # print out the last row in db
    # cur.execute('SELECT * FROM meta_score WHERE ROWID IN ( SELECT max( ROWID ) FROM meta_score )');
    conn.close()
    
# create 'app_store' table
def create_itunes_info_data_table(app_info):
    conn, cur = create_connection()
    # Create a brand new "itunes_info" table in db
    try:
        cur.execute('DROP TABLE itunes_info');
    except:
        print("Create info table for the first time")
    cur.execute('CREATE TABLE itunes_info (name TEXT, id INTEGER, rating FLOAT,\
                artistName TEXT, artistId INTEGER, price FLOAT)');
    cur.executemany('INSERT INTO itunes_info VALUES (?,?,?,?,?,?)', app_info);
    conn.commit()
    # Check table content:
    # cur.execute('SELECT * FROM itunes_info')
    # print(cur.fetchall())
    conn.close()


In [21]:
# db to Pandas Dataframe key = meta/info/review
def get_df(key):
    conn, cur = create_connection()
    key_convert = {'meta':'meta_score','info':'itunes_info','review':'itunes_review'}
    # print dataframe's head
    df = pd.read_sql_query(f"SELECT * FROM {key_convert[key]}", conn)
    conn.close()
    print(df.head())
    return df

In [4]:
# list of tuples: (app name,score)
meta = get_meta_score(5)
create_meta_data_table(meta)

# Use iTunes Search API to Get App info:

In [28]:
def search_url(name,country):
    # iTunes API requires replace from space to "+"
    name = name.replace(" ","+")
    return f'https://itunes.apple.com/search?term={name}&country={country}&entity=software'

def get_single_info(name):
    # get json of iTunes Search Report
    json = requests.get(search_url(name,"us")).json()
    # Iterate Thru for a single app with Exact Same Name
    for app in json['results']:
        if app['trackName'] == name:
            return(app['trackName'],app['trackId'],app['averageUserRating'],
                   app['artistName'],app['artistId'],app['price'])

def get_all_info(name_list):
    info_list = []
    for name in name_list:
        info_list.append(get_single_info(name))
    return info_list

# TESTING PART

In [ ]:
search_res = requests.get(search_url("Meteorfall: Journey","us"))
print(search_res.json())

In [27]:
df = get_df('meta')

                                           name  score
0                               World of Goo HD     96
1                                  World of Goo     96
2                               Super QuickHook     96
3  The World Ends with You: Solo Remix for iPad     95
4                           Meteorfall: Journey     95


0      96
1      96
2      96
3      95
4      95
       ..
495    85
496    85
497    85
498    85
499    85
Name: score, Length: 500, dtype: int64